# Demo — CNN + Majority Vote (19-slice OCT volume)

This notebook runs **inference only** for the baseline model:
- **Per-slice CNN inference**
- **Majority vote** across 19 slices

If you start from `.E2E` files, see the README section **Convert `.E2E` → PNG first**.


In [ ]:
# (Optional) Install dependencies (run once)
# If you already ran `pip install -r requirements.txt` in a terminal, you can skip this cell.

!pip -q install -r requirements.txt


In [ ]:
# Sanity check: you must run this notebook from the repository folder (TestAI)
import os
from pathlib import Path

print('Current working directory:', os.getcwd())
print('Has tools/download_checkpoints.py:', Path('tools/download_checkpoints.py').exists())
print('Has inference.py:', Path('inference.py').exists())
assert Path('tools/download_checkpoints.py').exists(), (
    'tools/download_checkpoints.py not found.\n'
    'Open Jupyter from the TestAI folder (the one containing README.md, tools/, models/, ...).'
)
assert Path('inference.py').exists(), 'inference.py not found — did you open the correct folder?'


## 1) CONFIG (edit only this cell)


In [ ]:
from pathlib import Path
import torch

# Folder that contains your PNG images.
# You can point to your folder containing all class folder (CHM/ Healthy/ USH2A/)
INPUT_DIR = Path(r"C:\\path\\to\\dataset_png")  # <-- CHANGE THIS

# Label order used during training
CLASS_NAMES = ["CHM", "Healthy", "USH2A"]

# Checkpoint filename (downloaded from GitHub Releases v1.0 into ./weights/)
WEIGHTS_PATH = Path("weights/cnn_resnet50_2025-10-20_best.pt")

# Device
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print('INPUT_DIR :', INPUT_DIR)
print('WEIGHTS   :', WEIGHTS_PATH)
print('DEVICE    :', DEVICE)
print('CLASSES   :', CLASS_NAMES)


## 2) Download checkpoints (from GitHub Releases)


In [ ]:
# This downloads all *.pt assets from Releases/v1.0 into ./weights/
# If you already downloaded them, the script will skip existing files.

!python tools/download_checkpoints.py


## 3) Run inference (CNN + vote)


In [ ]:
from inference import run_inference

df = run_inference(
    input_dir=INPUT_DIR,
    model_name="cnn_vote",
    device=DEVICE,
    class_names=CLASS_NAMES,
    weights_path=WEIGHTS_PATH,
)
df.head()


## 4) Save predictions


In [ ]:
Path('outputs').mkdir(parents=True, exist_ok=True)
out_path = Path('outputs/predictions_cnn_vote.csv')
df.to_csv(out_path, index=False)
print('Saved ->', out_path)
out_path
